#**1 Introduccion**

En el siguiente cuaderno se realizara el producto entre dos matrices cuadradas, el resultado del producto es una nueva matriz. Es decir:

<center>$\ MR= A x B =A (a(11)...a(1n)) *B(b(11)...b(1n)) = (a(11)b(11)+ ... +a(1n)b(1n))$</center>

Si bien el producto entre matrices tienen la propiedad de que el número de columnas de la matriz A tiene que ser igual al número de filas de la matriz B, en este ejercicio se opto por realizar el producto solo para matrices cuadradas.

En cuanto a la descripcion de la paralelizacion:

Como se ha definido cada bloque de forma bidimensional, se calculan índices de columnas y filas (después se convertirán a un índice lineal para hacer las operaciones).
Se lanzan threads, que en cada uno de ellos se generará el cálculo de su correspondiente elemento en la matriz resultante. Para poder generar éste resultado, es necesario hacer un ciclo hasta "n" para hacer la sumatoria de las multiplicaciones necesarias. Para que la asignacion de la sumatoria en la matriz resultante sea correcta se sincronizan los hilos con __syncthreads() [1].


#**2 Armando el ambiente**

Instala en el cuaderno el modulo de CUDA de python

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 9.9MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=45247d9ecb7bbebca337e28f14864f001ea485868fc7852040bddb1555d62b68
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=ca08b5a3829cbec5012ec4fb549207259ca18c0aa81249958353cda9373e65a8
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools



#**3 Desarrollo**

In [ ]:
# ------------------------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

f =   3#@param {type: "number"}
c= 3#@param {type: "number"}

# ------------------------------------------------------------

import pycuda.driver as cuda
import pycuda.autoinit
from   pycuda.compiler import SourceModule

import numpy
import copy

matriz_cpu = numpy.random.random((f,c)) *10
matriz_cpu = matriz_cpu.astype(numpy.int32())

matriz1 = numpy.random.random((f,c)) *10
matriz1 = matriz1.astype(numpy.int32())

matriz_r = numpy.random.random((f,c)) *10
matriz_r = matriz_r.astype(numpy.int32())
 
matriz_gpu = cuda.mem_alloc(matriz_cpu.nbytes)
matriz1_gpu = cuda.mem_alloc(matriz1.nbytes)
matrizr_gpu = cuda.mem_alloc(matriz_r.nbytes)

cuda.memcpy_htod(matriz_gpu,matriz_cpu)
cuda.memcpy_htod(matriz1_gpu,matriz1)
cuda.memcpy_htod(matrizr_gpu,matriz_r)


#CPU - Defino la funcion kernel que ejecutará en GPU
module = SourceModule("""
__global__ void productoMatrices(int n, int *M1, int *M2, int *MR)
{
    int col = threadIdx.x + blockIdx.x*blockDim.x;
    int fila = threadIdx.y + blockIdx.y*blockDim.y;
    float suma=0;

    if(col<n && fila<n)
    {
       for(int k=0; k<n;k++){
         float a = M1[col*n+k];
         float b = M2[k*n+fila];
         suma+= a*b;
       }
       __syncthreads();
       MR[col*n+fila] = suma;
    }
}

""")

kernel = module.get_function("productoMatrices")

dim_hilo_x = 16
dim_bloque_x = numpy.int( (f+dim_hilo_x-1) / dim_hilo_x )

dim_hilo_y = 16
dim_bloque_y = numpy.int( (c+dim_hilo_y-1) / dim_hilo_y )

print( "Thread x: ", dim_hilo_x, ", Bloque x:", dim_bloque_x )
print( "Thread y: ", dim_hilo_y, ", Bloque y:", dim_bloque_y )

kernel( numpy.int32(f), matriz_gpu, matriz1_gpu, matrizr_gpu, block=( dim_hilo_x, dim_hilo_y, 1 ),grid=(dim_bloque_x, dim_bloque_y,1) )
cuda.memcpy_dtoh(matriz_r,matrizr_gpu)

print("Matriz 1: ")
print(matriz_cpu)
print("Matriz 2:")
print(matriz1)
print("Resultado de la suma de matrices")
print(matriz_r)


Thread x:  16 , Bloque x: 1
Thread y:  16 , Bloque y: 1
Matriz 1: 
[[7 1 2]
 [0 8 3]
 [6 4 8]]
Matriz 2:
[[3 2 6]
 [3 6 9]
 [9 4 4]]
Resultado de la suma de matrices
[[ 42  28  59]
 [ 51  60  84]
 [102  68 104]]


#**4 Tabla de pasos**

 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de las matrices en Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  numpy.random.randn( (n,n ) | Inicializa los Matrices.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria matrizr_gpu a CPU memoria matriz_r.
CPU      |  print()               | Informo los resultados.


#**5 Conclusion**

Se puede observar que en el producto de matrices en la GPU, debido a que un único thread se utiliza para calcular el valor de MR(i,j), la complejidad resultante[2] es O(N) que acomparacion del producto en el CPU [3] resulta ser mucho mas óptimo y por lo tanto mas rápido.


#**6 Bibliografia**

[1] Warps, Blocks, and Synchronization: [PDF](https://www.math.wsu.edu/math/kcooper/CUDA/13CUDAblock.pdf)

[2] Complejidad Computacional:[PDF](https://www.frlp.utn.edu.ar/materias/algoritmos/GUIACOMPLEJIDADDEALGORITMOS.pdf)

[3] Ejercicio Secuencial: [GITHUB](https://)